# 3가지 주요 Tokenizer 직접 구현
1. BPE(Byte-Pair Encoding)
2. WordPiece
3. Unigram (SentencePiece 스타일)


| 모델            | 특징                      | 한국어에서 경향               |
| ------------- | ----------------------- | ---------------------- |
| **BPE**       | 가장 자주 나오는 문자쌍 병합        | 적당한 길이의 subword        |
| **WordPiece** | likelihood가 가장 높은 분해 선택 | ##붙은 subword 많음        |
| **Unigram**   | 전체 단어를 확률적으로 선택         | 가장 자연스러운 분절이 나오는 경우 많음 |


In [2]:
from tokenizers import Tokenizer
from tokenizers.models import BPE, WordPiece, Unigram
from tokenizers.trainers import BpeTrainer, WordPieceTrainer, UnigramTrainer
from tokenizers.pre_tokenizers import Whitespace

In [3]:
def train_tokenizer(corpus, model, vocab_size=50):
    if model == "bpe":
        tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
        trainer = BpeTrainer(vocab_size=vocab_size, special_tokens=["[UNK]"])
    elif model == "wordpiece":
        tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
        trainer = WordPieceTrainer(vocab_size=vocab_size, special_tokens=["[UNK]"])
    elif model == "unigram":
        tokenizer = Tokenizer(Unigram())
        trainer = UnigramTrainer(vocab_size=vocab_size, unk_token="[UNK]", special_tokens=["[UNK]"])
    else:
        raise ValueError(f"Unknown model:{model}")
        
    tokenizer.pre_tokenizer = Whitespace()
    tokenizer.train_from_iterator(corpus, trainer)
    return tokenizer

def test_sentence(sentence):
    print("===== BPE =====")
    enc = bpe_tokenizer.encode(sentence)
    print(f"Tokens:{enc.tokens}")
    print(f"Tokens:{enc.ids}")
    
    print("\n===== WordPiece =====")
    enc = wordpiece_tokenizer.encode(sentence)
    print(f"Tokens:{enc.tokens}")
    print(f"Tokens:{enc.ids}")
    
    print("\n===== Unigram =====")
    enc = unigram_tokenizer.encode(sentence)
    print(f"Tokens:{enc.tokens}")
    print(f"Tokens:{enc.ids}")

In [4]:
corpus = [
    "나는 밥을 먹었다",
    "나는 밥을 먹고 학교에 갔다",
    "학교에서 친구를 만났다"
]

bpe_tokenizer = train_tokenizer(corpus, "bpe")
wordpiece_tokenizer = train_tokenizer(corpus, "wordpiece")
unigram_tokenizer = train_tokenizer(corpus, "unigram")

In [5]:
test_sentence("나는 친구와 학교에서 밥을 먹었다")

===== BPE =====
Tokens:['나는', '친', '구', '[UNK]', '학교에서', '밥을', '먹었다']
Tokens:[20, 17, 4, 0, 30, 21, 31]

===== WordPiece =====
Tokens:['나는', '[UNK]', '학교에서', '밥을', '먹었다']
Tokens:[30, 0, 39, 31, 41]

===== Unigram =====
Tokens:['나', '는', '친', '구', '와', '학교에', '서', '밥', '을', '먹', '었', '다']
Tokens:[6, 4, 14, 18, 0, 7, 19, 5, 2, 3, 16, 1]


In [6]:
test_sentence("밥을 학교 가게에서 먹었다")

===== BPE =====
Tokens:['밥을', '학', '교', '[UNK]', '[UNK]', '에', '서', '먹었다']
Tokens:[21, 18, 3, 0, 0, 15, 13, 31]

===== WordPiece =====
Tokens:['밥을', '학교', '[UNK]', '먹었다']
Tokens:[31, 32, 0, 41]

===== Unigram =====
Tokens:['밥', '을', '학', '교', '가게', '에', '서', '먹', '었', '다']
Tokens:[5, 2, 8, 10, 0, 9, 19, 3, 16, 1]
